In [1]:
fp = f"/Users/gugujixiao/workspace/project/HWMemory/Code/HW-Memory/example/old_example/forTest.asm"

In [2]:
from sample.cfg import proc_identify
from sample.isa import Instruction, AddrMode
from typing import Tuple
from sample.read_asm import AsmFileReader, StatementType

reader =AsmFileReader(fp)

statements = list()
temp_stat = list()
for s in reader.statements:
    s: Tuple[StatementType, tuple]
    if s[0] == StatementType.Instruction:
        statements.append((s[0], Instruction(s[1])))
        temp_stat.append(Instruction(s[1]))
    elif s[0] == StatementType.SubProcedure:
        statements.append(s)

#for i in temp_stat:
#    booltemp = i.is_ls
#    if booltemp:
#        if i.ls_addr_mode == AddrMode.ImmeAft:
#            print(i.ls_reg_traget)

procs = proc_identify(statements)
for proc in procs:
    print(proc.name, proc.beg_addr.hex_str())
    if False and proc.name == 'main':
        for inst in proc.instruction:
            print(inst.addr.hex_str(), inst.opcode, inst.name, inst.operands, inst.branch_info)
        print()

_init 00000000000005e8
.plt 0000000000000600
__cxa_finalize@plt 0000000000000620
__libc_start_main@plt 0000000000000630
__gmon_start__@plt 0000000000000640
abort@plt 0000000000000650
printf@plt 0000000000000660
_start 0000000000000670
call_weak_fn 00000000000006a8
deregister_tm_clones 00000000000006c0
register_tm_clones 00000000000006f0
__do_global_dtors_aux 0000000000000730
frame_dummy 0000000000000778
main 000000000000077c
Duo 00000000000008a4
square 00000000000008c4
cube 00000000000008e4
__libc_csu_init 0000000000000910
__libc_csu_fini 0000000000000990
_fini 0000000000000994


In [3]:
from graphviz import Digraph
from sample.cfg import draw_proc, find_cycle, has_cycle, proc_draw_edges

proc_draw_edges(procs)

is_cycle = has_cycle(procs)
if is_cycle:
    c = [p.name for p in find_cycle(procs)]
    raise RuntimeError("Loop between procedures is not allowed: {}.".format(c))

g: Digraph = draw_proc(procs)
g.render(filename='procedures', directory='./output', format='svg')

'output/procedures.svg'

In [4]:
from sample.cfg import CallGraph

call_graph = CallGraph(procs)

g = call_graph.draw_graph()
g.render(filename='call_graph', directory='./output', format='svg')

'output/call_graph.svg'

In [5]:
from sample.cfg import TCfg
from sample.rw_condition import hw_CFG_Node

tcfg = TCfg(call_graph)
tcfg_nodes = tcfg.all_nodes
tcfg_edges = tcfg.edges

for node in tcfg_nodes:

    node.set_rw_condition()


#for node in tcfg_nodes:
#    node.set_rw_condition()
    #print(node.name, node.base_proc.name, node.inst_range)
    #print(node.name, node.base_proc.name)
    #print("出边")
    #for i in node.from_edge:
    #    print(i.dst.name)
    #print("入边")
    #for i in node.to_edge:
    #    print(i.src.name)
tempbool = True

#for node in tcfg_nodes:
#    node.set_rw_value()
while tempbool:
    no_end = False
    for node in tcfg_nodes:
        node.set_rw_value()
        if not node.no_point_other:
            no_end = True
    tempbool = no_end
        
for node in tcfg_nodes:
    print(node.name,node.node_value)

#for i in hw_cfg_node_list:
    '''print(i.node_self.name)
    print("来的点")
    for j in i.node_from:
        print(j.name)
    print("出去的点")
    for k in i.node_to:
        print(k.name)
    print("指令")
    for a in i.instr:
        print(a.tokens)
    print()'''

    #if i.is_end:
    #print(i.node_self.name,i.node_from_num)

#for edge in tcfg_edges:
#    print(edge.src.name, edge.dst.name, edge.kind.name)

g = tcfg.draw_graph()
g.render(filename='tcfg', directory='./output', format='svg')

n0 1
n1 1.0
n2 1.0
n3 1.0
n4 1.0
n5 1.0
n6 1.0
n7 0.5
n8 0.5
n9 1.0
n10 1.0
n11 1.0
n12 1.0
n13 1.0
n14 1.0
n15 1.0
n16 0.5
n17 0.5


'output/tcfg.svg'